# CS492 전산학특강<인공지능 산업 및 스마트에너지>
## Deep Learning Practice 
#### Prof. Ho-Jin Choi
#### School of Computing, KAIST

---

### 2-3. Overfitting and how to fight it

In the previous example, we saw that the accuracy of our model on the validation data would peak after training for a number of epochs, and would then start decreasing or being fixed.

In other words, our model would _overfit to the training data_. Learning how to deal with overfitting is important. Although it's often possible to achieve high accuracy on the training set, what we really want is to develop models that generalize well to a testing set (or data they haven't seen before). If you train for too long though, the model will start to overfit and learn patterns from the training data that don't generalize to the test data. We need to strike a balance. Understanding how to train for an appropriate number of epochs as we'll explore below is a useful skill.

To prevent overfitting, the best solution is to use more training data. A model trained on more data will naturally generalize better. When that is no longer possible, the next best solution is to use techniques like _regularization_. These place constraints on the quantity and type of information your model can store. If a network can only afford to memorize a small number of patterns, the optimization process will force it to focus on the most prominent patterns, which have a better chance of generalizing well.

In this part, we'll explore two common regularization techniques, weight regularization and dropout, and use them to improve our model.

First, we'll set up the environment as before.

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
print('Tensorflow: ', tf.__version__)

import numpy as np
import matplotlib.pyplot as plt

Tensorflow:  2.0.0


#### Load the IMDB dataset

In [2]:
NUM_WORDS = 10000

(train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.imdb.load_data(num_words=NUM_WORDS)

def multi_hot_sequences(sequences, dimension):
    # Create an all-zero matrix of shape (len(sequences), dimension)
    results = np.zeros((len(sequences), dimension))
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0  # set specific indices of results[i] to 1s
    return results

train_data = multi_hot_sequences(train_data, dimension=NUM_WORDS)
test_data = multi_hot_sequences(test_data, dimension=NUM_WORDS)

#### Create a validation set

#### Demonstrate overfitting by model size

The simplest way to prevent overfitting is to reduce the size of the model, i.e. the number of learnable parameters in the model. In deep learning, the number of learnable parameters in a model is often referred to as the model's **"_capacity_"**. Intuitively, a model with more parameters will have more "memorization capacity" and therefore will be able to easily learn a perfect dictionary-like mapping between training samples and their targets, **a mapping without any generalization power, but this would be useless when making predictions on previously unseen data**.

On the other hand, if the **network has limited memorization resources**, it will not be able to learn the mapping as easily. To minimize its loss, **it will have to learn compressed representations that have more predictive power.** At the same time, if you make your model too small, **it will have difficulty fitting to the training data.** There is a balance between "too much capacity" and "not enough capacity".

To find an appropriate model size empirically, it's best to start with relatively few layers and parameters, then begin increasing the size of the layers or adding new layers until you see diminishing returns on the validation loss. Let's try this on our movie review classification network.

We'll create a simple model using only Dense layers as a baseline, then create smaller and larger versions, and compare them.

#### Create a baseline model

In [ ]:
baseline_model = tf.keras.Sequential([
    # TODO: add a first dense layer with 16 nodes applying ReLU activation function

    # TODO: add a second dense layer with 16 nodes applying ReLU activation function
    
    # TODO: add a last dense layer with only 1 node applying Softmax activation function
])

baseline_model.summary()



In [ ]:
# Configure the model with 
#                optimize: Adam optimizer
#                loss func: Binary crossentropy
#                metrics: Accuracy
baseline_model.compile(


)

In [ ]:
# Train the baseline model with partial training data and validation data we made 
baseline_history = baseline_model.fit(



)


#### Create a smaller model
Let's create a model with less hidden units to compare against the baseline model that we just created:

In [ ]:
smaller_model = tf.keras.Sequential([
    # TODO: add a first dense layer with 4 nodes applying ReLU activation function

    # TODO: add a second dense layer with 4 nodes applying ReLU activation function
    
    # TODO: add a last dense layer with only 1 node applying Softmax activation function
])

smaller_model.summary()

In [ ]:
# Configure the model with 
#                optimize: Adam optimizer
#                loss func: Binary crossentropy
#                metrics: Accuracy
smaller_model.compile(


)

In [ ]:
# Train the smaller model with partial training data and validation data we made 
smaller_history = smaller_model.fit(
    
    
)

#### Create a bigger model
As an exercise, you can create an even larger model, and see how quickly it begins overfitting. Next, let's add to this benchmark a network that has much more capacity, far more than the problem would warrant:

In [ ]:
bigger_model = tf.keras.Sequential([
    # TODO: add a first dense layer with 512 nodes applying ReLU activation function
    
    # TODO: add a second dense layer with 512 nodes applying ReLU activation function

    # TODO: add a last dense layer with only 1 node applying Softmax activation function
])

bigger_model.summary()

In [ ]:
# Configure the model with 
#                optimize: Adam optimizer
#                loss func: Binary crossentropy
#                metrics: Accuracy
bigger_model.compile(


)

In [ ]:
# Train the bigger model with partial training data and validation data we made 
bigger_history = bigger_model.fit(



)

#### Plot the training and validation loss
The solid lines show the training loss, and the dashed lines show the validation loss (remember: a lower validation loss indicates a better model). Here, the smaller network begins overfitting later than the baseline model (after 6 epochs rather than 4) and its performance degrades much more slowly once it starts overfitting.

In [ ]:
history_dict = baseline_history.history
history_dict.keys()

In [ ]:
def plot_history(histories, key='loss'):
    plt.figure(figsize=(16,10))
    
    for name, history in histories:
        val = plt.plot(history.epoch, history.history['val_'+key],
                       '--', label=name.title()+' Val')
        plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
                 label=name.title()+' Train')
        
    plt.xlabel('Epochs')
    plt.ylabel(key.replace('_',' ').title())
    plt.legend()
    
    plt.xlim([0,max(history.epoch)])

In [ ]:
plot_history([('baseline', baseline_history),
              ('smaller', smaller_history),
              ('bigger', bigger_history)])

Notice that the larger network begins overfitting almost right away, after just one epoch, and overfits much more severely. **The more capacity the network has, the quicker it will be able to model the training data (resulting in a low training loss)**, but the more susceptible it is to overfitting (resulting in a large difference between the training and validation loss).

#### Solution 1: Add weight regularization

Given some training data and a network architecture, there are multiple sets of weights values (multiple models) that could explain the data, and simpler models are less likely to overfit than complex ones. A "simple model" in this context is a model where the distribution of parameter values has less entropy (or a model with fewer parameters altogether, as we saw in the section above). 

Thus a common way to mitigate overfitting is to **put constraints on the complexity of a network by forcing its weights only to take small values**, which **makes the distribution of weight values more "regular".** This is called **"_weight regularization_"**, and it is done by adding to the loss function of the network a cost associated with having large weights. This cost comes in two flavors:

 - [L1 regularization](https://developers.google.com/machine-learning/glossary/#L1_regularization), where the cost added is proportional to the absolute value of the weights coefficients (i.e. to what is called the "L1 norm" of the weights).

 - [L2 regularization](https://developers.google.com/machine-learning/glossary/#L2_regularization), where the cost added is proportional to the square of the value of the weights coefficients (i.e. to what is called the squared "L2 norm" of the weights). L2 regularization is also called weight decay in the context of neural networks. Don't let the different name confuse you: weight decay is mathematically the exact same as L2 regularization.
 
In `tf.keras`, weight regularization is added by passing weight regularizer instances to layers as keyword arguments. Let's add L2 weight regularization now.

In [ ]:
l2_model = tf.keras.Sequential([
    # TODO: add a first dense layer with 16 nodes applying ReLU activation function 
    #                   with L2 regularization term 

    # TODO: add a second dense layer with 16 nodes applying ReLU activation function
    #                   with L2 regularization term 

    
    # TODO: add a last dense layer with only 1 node applying Softmax activation function
])

l2_model.summary()

In [ ]:
# Configure the model with 
#                optimize: Adam optimizer
#                loss func: Binary crossentropy
#                metrics: Accuracy
l2_model.compile(


)

l2(0.001) means that every coefficient in the weight matrix of the layer will add 0.001 * weight_coefficient_value**2 to the total loss of the network. Note that because this penalty is only added at training time, the loss for this network will be much higher at training than at test time.

Here's the impact of our L2 regularization penalty:

In [ ]:
# Train the l2 model with partial training data and validation data we made 

l2_model_history = l2_model.fit(


)

In [ ]:
plot_history([('baseline', baseline_history),
              ('l2', l2_model_history)])

As you can see, the L2 regularized model has become much more resistant to overfitting than the baseline model, even though both models have the same number of parameters.

#### Solution 2: Add dropout

Dropout is one of the most effective and most commonly used regularization techniques for neural networks. Dropout, applied to a layer, consists of randomly "dropping out" (i.e. set to zero) a number of output features of the layer during training. 

<img src="images/dropout.png"  width="500">

The "dropout rate" is the fraction of the features that are being zeroed-out; it is usually set between 0.2 and 0.5. At test time, no units are dropped out, and instead the layer's output values are scaled down by a factor equal to the dropout rate, so as to balance for the fact that more units are active than at training time.

In tf.keras you can introduce dropout in a network via the Dropout layer, which gets applied to the output of layer right before.

Let's add two Dropout layers in our IMDB network to see how well they do at reducing overfitting:

In [ ]:
dp_model = tf.keras.Sequential([
    # TODO: add a first dense layer with 16 nodes applying ReLU activation function 

    # TODO: add a dropout layer with 0.5 dropout rate

    # TODO: add a second dense layer with 16 nodes applying ReLU activation function

    # TODO: add a dropout layer with 0.5 dropout rate

    # TODO: add a last dense layer with only 1 node applying Softmax activation function

])

dp_model.summary()

In [ ]:
# Configure the model with 
#                optimize: Adam optimizer
#                loss func: Binary crossentropy
#                metrics: Accuracy
dp_model.compile(


)

In [ ]:
# Train the dropout model with partial training data and validation data we made 

dp_model_history = dp_model.fit(



)

In [ ]:
plot_history([('baseline', baseline_history),
              ('dropout', dp_model_history)])

Adding dropout is a clear improvement over the baseline model.

To recap: here the most common ways to prevent overfitting in neural networks:

- Get more training data.
- Reduce the capacity of the network.
- Add weight regularization.
- Add dropout.

And two important approaches not covered in this guide are data-augmentation and batch normalization.